In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
import os
os.chdir('/content/drive/MyDrive/Projects/reddit-vote-predictor')
import sys
if 'psaw' not in sys.modules:
  print('Installing psaw')
  os.system('pip install psaw')
from datetime import datetime
from scripts.RedditScrape import * # imports a class `RedditScrape`

Installing psaw


In [ ]:
# Set date range from which to extract data for two subreddits
# NOTE: ater running the data scraping and cleaning pipeline, one year of data
# resulted in around 1000 images, so we'll add time to the date range
# 10 years from the start of 2011 to the end of 2020
start_ = datetime(2011, 1, 1)
end_ = datetime(2020, 12, 31)
# Run the scraping pipeline using custom RedditScrape class imported above
cat = RedditScrape('cat', start_, end_)
cat.scrape_posts()
cat.posts.to_csv('data/tbl/cat_posts.csv', index=False)
dog = RedditScrape('dog', start_, end_)
dog.scrape_posts()
dog.posts.to_csv('data/tbl/dog_posts.csv', index=False)

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")
